<a href="https://colab.research.google.com/github/sneymz00/42/blob/master/SQL_3y4%2BCorrecci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 REPASO SQL BÁSICO - ORDER BY Y JOINS
**Ejercicios para dominar ordenamiento y uniones de tablas**

---

## 🎯 OBJETIVOS DE ESTE REPASO

Este notebook se enfoca en dos conceptos fundamentales:

### **📊 EJERCICIO 3: ORDER BY**
- ✅ **ORDER BY básico** - Una columna ASC/DESC
- ✅ **ORDER BY múltiple** - Múltiples columnas
- ✅ **ORDER BY + LIMIT** - Top N resultados
- ✅ **Combinaciones** con WHERE y otras cláusulas

### **🔗 EJERCICIO 4: JOINs BÁSICOS**
- ✅ **INNER JOIN** - Solo registros que coinciden
- ✅ **LEFT JOIN** - Incluir registros sin coincidencias
- ✅ **JOINs con cálculos** - Agregar computaciones
- ✅ **JOINs múltiples** - 3+ tablas

### **🏆 EJERCICIO FINAL**
- ✅ **Integración completa** - Combina todo lo aprendido
- ✅ **Casos de negocio** - Problemas reales de análisis
- ✅ **Nivel medio** - Un paso más allá de lo básico

### 📋 **Base de datos:**
- **4 tablas relacionadas**: categorias, clientes, productos, ventas
- **Datos realistas**: tienda online con categorías, clientes y ventas
- **Relaciones claras**: foreign keys bien definidas

---

## 🔧 SETUP INICIAL

In [ ]:
# Configurar entorno para ORDER BY y JOINs
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Crear conexión
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

print("🚀 REPASO ORDER BY Y JOINS INICIADO")
print("="*50)
print("✅ SQLite configurado")
print("📊 Pandas listo")
print("📈 Matplotlib y Seaborn preparados")

# Función mejorada para ejecutar SQL
def sql(query, mostrar=True, titulo=None):
    """Ejecuta SQL con título opcional"""
    try:
        if titulo:
            print(f"\n🔍 {titulo}")
            print("-" * len(titulo))

        df = pd.read_sql_query(query, conn)

        if mostrar:
            if len(df) > 0:
                print(df.to_string(index=False))
            else:
                print("⚠️ No hay resultados")
        return df
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return None

# Función para verificar ejercicios
def verificar_ejercicio(numero, tu_query, solucion_query):
    """Verifica si tu consulta coincide con la solución"""
    print(f"\n✅ VERIFICACIÓN EJERCICIO {numero}")
    print("=" * 40)

    try:
        tu_resultado = pd.read_sql_query(tu_query, conn)
        solucion = pd.read_sql_query(solucion_query, conn)

        # Comparar estructura y contenido
        if list(tu_resultado.columns) != list(solucion.columns):
            print("⚠️ Las columnas no coinciden")
            print(f"Tus columnas: {list(tu_resultado.columns)}")
            print(f"Esperadas: {list(solucion.columns)}")
        elif len(tu_resultado) != len(solucion):
            print(f"⚠️ Número de filas diferente: {len(tu_resultado)} vs {len(solucion)}")
        else:
            print("✅ ¡PERFECTO! Tu consulta es correcta")
            print(f"📊 Resultados: {len(tu_resultado)} filas")

        print("\n📋 Tu resultado:")
        print(tu_resultado.to_string(index=False))

    except Exception as e:
        print(f"❌ Error en tu consulta: {str(e)}")
        print("\n💡 Solución correcta:")
        sql(solucion_query)

print("\n🛠️ Funciones disponibles:")
print("   - sql(query, titulo='descripción') → ejecutar SQL")
print("   - verificar_ejercicio(num, tu_query, solucion) → verificar respuesta")
print("\n🎊 ¡Listo para practicar ORDER BY y JOINs!")

## 🗄️ CREAR BASE DE DATOS

Vamos a crear una base de datos más completa con relaciones entre tablas:

In [ ]:
# Crear esquema de tablas relacionadas
cursor.executescript("""
-- Tabla de categorías
CREATE TABLE categorias (
    id INTEGER PRIMARY KEY,
    nombre VARCHAR(30),
    descripcion VARCHAR(100)
);

-- Tabla de clientes
CREATE TABLE clientes (
    id INTEGER PRIMARY KEY,
    nombre VARCHAR(50),
    apellido VARCHAR(50),
    email VARCHAR(80),
    ciudad VARCHAR(30),
    edad INTEGER,
    fecha_registro DATE
);

-- Tabla de productos
CREATE TABLE productos (
    id INTEGER PRIMARY KEY,
    nombre VARCHAR(60),
    categoria_id INTEGER,
    precio DECIMAL(8,2),
    stock INTEGER,
    activo BOOLEAN DEFAULT 1,
    FOREIGN KEY (categoria_id) REFERENCES categorias(id)
);

-- Tabla de ventas
CREATE TABLE ventas (
    id INTEGER PRIMARY KEY,
    cliente_id INTEGER,
    producto_id INTEGER,
    cantidad INTEGER,
    precio_unitario DECIMAL(8,2),
    fecha_venta DATE,
    vendedor VARCHAR(50),
    FOREIGN KEY (cliente_id) REFERENCES clientes(id),
    FOREIGN KEY (producto_id) REFERENCES productos(id)
);
""")

print("🏗️ Esquema de base de datos creado")
print("\n📋 Tablas con relaciones:")
print("   📁 categorias (5 registros)")
print("   👥 clientes (12 registros)")
print("   📦 productos (15 registros) → categoria_id")
print("   💰 ventas (20 registros) → cliente_id, producto_id")

In [ ]:
# Insertar datos realistas
cursor.executescript("""
-- Categorías
INSERT INTO categorias VALUES
(1, 'Electrónicos', 'Dispositivos electrónicos y gadgets'),
(2, 'Ropa', 'Vestimenta y accesorios de moda'),
(3, 'Hogar', 'Artículos para el hogar y decoración'),
(4, 'Deportes', 'Equipos y ropa deportiva'),
(5, 'Libros', 'Libros físicos y digitales');

-- Clientes
INSERT INTO clientes VALUES
(1, 'Ana', 'García', 'ana.garcia@email.com', 'Madrid', 28, '2023-01-15'),
(2, 'Carlos', 'López', 'carlos.lopez@email.com', 'Barcelona', 34, '2023-01-20'),
(3, 'María', 'Ruiz', 'maria.ruiz@email.com', 'Valencia', 25, '2023-02-01'),
(4, 'José', 'Martín', 'jose.martin@email.com', 'Sevilla', 42, '2023-02-10'),
(5, 'Laura', 'Torres', 'laura.torres@email.com', 'Madrid', 31, '2023-02-15'),
(6, 'Pedro', 'Sánchez', 'pedro.sanchez@email.com', 'Barcelona', 29, '2023-03-01'),
(7, 'Carmen', 'Vega', 'carmen.vega@email.com', 'Valencia', 37, '2023-03-05'),
(8, 'Antonio', 'Díaz', 'antonio.diaz@email.com', 'Madrid', 33, '2023-03-10'),
(9, 'Isabel', 'Moreno', 'isabel.moreno@email.com', 'Sevilla', 26, '2023-03-15'),
(10, 'Francisco', 'Jiménez', 'francisco.jimenez@email.com', 'Barcelona', 39, '2023-03-20'),
(11, 'Rosa', 'Herrera', 'rosa.herrera@email.com', 'Valencia', 35, '2023-04-01'),
(12, 'Manuel', 'Castro', 'manuel.castro@email.com', 'Madrid', 41, '2023-04-05');

-- Productos
INSERT INTO productos VALUES
(1, 'iPhone 15', 1, 999.99, 25, 1),
(2, 'Samsung Galaxy S24', 1, 899.99, 30, 1),
(3, 'Laptop Dell XPS', 1, 1299.99, 15, 1),
(4, 'Camiseta Nike', 2, 29.99, 100, 1),
(5, 'Jeans Levis', 2, 79.99, 50, 1),
(6, 'Zapatillas Adidas', 2, 89.99, 75, 1),
(7, 'Lámpara LED', 3, 49.99, 40, 1),
(8, 'Sofá 3 plazas', 3, 599.99, 8, 1),
(9, 'Mesa comedor', 3, 299.99, 12, 1),
(10, 'Balón fútbol', 4, 24.99, 60, 1),
(11, 'Raqueta tenis', 4, 149.99, 20, 1),
(12, 'Bicicleta montaña', 4, 399.99, 10, 1),
(13, 'Harry Potter', 5, 19.99, 80, 1),
(14, 'Don Quijote', 5, 15.99, 45, 1),
(15, 'Cien años soledad', 5, 18.99, 35, 1);
""")

print("📊 Datos básicos insertados...")

In [ ]:
# Insertar ventas (la tabla más compleja)
cursor.execute("""
INSERT INTO ventas VALUES
(1, 1, 1, 1, 999.99, '2024-01-15', 'Juan Pérez'),
(2, 2, 4, 2, 29.99, '2024-01-20', 'Ana López'),
(3, 3, 13, 1, 19.99, '2024-01-25', 'Juan Pérez'),
(4, 1, 7, 1, 49.99, '2024-02-01', 'Carlos Ruiz'),
(5, 4, 2, 1, 899.99, '2024-02-05', 'Ana López'),
(6, 5, 10, 3, 24.99, '2024-02-10', 'Juan Pérez'),
(7, 6, 5, 1, 79.99, '2024-02-15', 'Carlos Ruiz'),
(8, 2, 14, 2, 15.99, '2024-02-20', 'Ana López'),
(9, 7, 8, 1, 599.99, '2024-03-01', 'Juan Pérez'),
(10, 8, 6, 1, 89.99, '2024-03-05', 'Carlos Ruiz'),
(11, 3, 11, 1, 149.99, '2024-03-10', 'Ana López'),
(12, 9, 15, 1, 18.99, '2024-03-15', 'Juan Pérez'),
(13, 10, 3, 1, 1299.99, '2024-03-20', 'Carlos Ruiz'),
(14, 4, 12, 1, 399.99, '2024-03-25', 'Ana López'),
(15, 11, 9, 1, 299.99, '2024-03-30', 'Juan Pérez'),
(16, 5, 13, 2, 19.99, '2024-04-01', 'Carlos Ruiz'),
(17, 12, 4, 1, 29.99, '2024-04-05', 'Ana López'),
(18, 6, 7, 2, 49.99, '2024-04-10', 'Juan Pérez'),
(19, 8, 10, 1, 24.99, '2024-04-15', 'Carlos Ruiz'),
(20, 9, 14, 1, 15.99, '2024-04-20', 'Ana López')
""")

conn.commit()
print("✅ Todas las ventas insertadas")

# Verificar datos
print("\n🔍 Verificación final:")
sql("""
SELECT 'Categorías' AS tabla, COUNT(*) AS registros FROM categorias
UNION ALL SELECT 'Clientes', COUNT(*) FROM clientes
UNION ALL SELECT 'Productos', COUNT(*) FROM productos
UNION ALL SELECT 'Ventas', COUNT(*) FROM ventas
""", titulo="Resumen de datos")

## 👀 EXPLORAR LAS RELACIONES

Antes de empezar, veamos cómo se relacionan nuestras tablas:

In [ ]:
print("📁 CATEGORÍAS:")
sql("SELECT * FROM categorias")

print("\n📦 PRODUCTOS (primeros 5):")
sql("SELECT id, nombre, categoria_id, precio, stock FROM productos LIMIT 5")

In [ ]:
print("👥 CLIENTES (primeros 5):")
sql("SELECT id, nombre, apellido, ciudad, edad FROM clientes LIMIT 5")

print("\n💰 VENTAS (primeras 5):")
sql("SELECT id, cliente_id, producto_id, cantidad, precio_unitario, fecha_venta FROM ventas LIMIT 5")

---

# 📊 EJERCICIO 3: ORDER BY - ORDENAR RESULTADOS
**20 ejercicios para dominar el ordenamiento**

## 🎯 PARTE A: ORDER BY BÁSICO

### Ejercicio 3.1: Ordenar clientes por nombre

In [ ]:
# EJERCICIO 3.1: Muestra todos los clientes ordenados por nombre alfabéticamente
# Columnas: nombre, apellido, ciudad

print("📝 EJERCICIO 3.1: Clientes ordenados por nombre")

# Tu respuesta aquí:
mi_consulta_3_1 = ""

if mi_consulta_3_1:
    sql(mi_consulta_3_1, titulo="Mi resultado")
else:
    print("✍️ Escribe tu consulta en mi_consulta_3_1")
    print("💡 PISTA: ORDER BY nombre")

In [ ]:
# Ver solución del ejercicio 3.1
solucion_3_1 = "SELECT nombre, apellido, ciudad FROM clientes ORDER BY nombre"
sql(solucion_3_1, titulo="✅ SOLUCIÓN EJERCICIO 3.1")

### Ejercicio 3.2: Productos por precio (descendente)

In [ ]:
# EJERCICIO 3.2: Muestra productos ordenados por precio de mayor a menor
# Columnas: nombre, precio

print("📝 EJERCICIO 3.2: Productos por precio (mayor a menor)")

mi_consulta_3_2 = ""

if mi_consulta_3_2:
    sql(mi_consulta_3_2)
else:
    print("✍️ Escribe tu consulta")
    print("💡 PISTA: ORDER BY precio DESC")

In [ ]:
# Solución 3.2
solucion_3_2 = "SELECT nombre, precio FROM productos ORDER BY precio DESC"
sql(solucion_3_2, titulo="✅ SOLUCIÓN EJERCICIO 3.2")

### Ejercicio 3.3: TOP 5 productos más caros

In [ ]:
# EJERCICIO 3.3: Muestra los 5 productos más caros
# Columnas: nombre, precio

print("📝 EJERCICIO 3.3: Top 5 productos más caros")

mi_consulta_3_3 = ""

if mi_consulta_3_3:
    sql(mi_consulta_3_3)
else:
    print("✍️ Escribe tu consulta")
    print("💡 PISTA: ORDER BY precio DESC LIMIT 5")

In [ ]:
# Solución 3.3
solucion_3_3 = "SELECT nombre, precio FROM productos ORDER BY precio DESC LIMIT 5"
sql(solucion_3_3, titulo="✅ SOLUCIÓN EJERCICIO 3.3")

## 🎯 PARTE B: ORDER BY MÚLTIPLE

### Ejercicio 3.4: Ordenar por ciudad y luego por nombre

In [ ]:
# EJERCICIO 3.4: Clientes ordenados por ciudad y luego por nombre
# Columnas: nombre, apellido, ciudad

print("📝 EJERCICIO 3.4: Ordenar por ciudad y luego por nombre")

mi_consulta_3_4 = ""

if mi_consulta_3_4:
    sql(mi_consulta_3_4)
else:
    print("✍️ Escribe tu consulta")
    print("💡 PISTA: ORDER BY ciudad, nombre")

In [ ]:
# Solución 3.4
solucion_3_4 = "SELECT nombre, apellido, ciudad FROM clientes ORDER BY ciudad, nombre"
sql(solucion_3_4, titulo="✅ SOLUCIÓN EJERCICIO 3.4")

---

# 🔗 EJERCICIO 4: JOINS BÁSICOS - UNIR TABLAS
**25 ejercicios para dominar las uniones**

## 🎯 PARTE A: INNER JOIN BÁSICO

### Ejercicio 4.1: Productos con su categoría

In [ ]:
# EJERCICIO 4.1: Muestra productos con el nombre de su categoría
# Columnas: nombre_producto, precio, nombre_categoria

print("📝 EJERCICIO 4.1: Productos con su categoría")

mi_consulta_4_1 = ""

if mi_consulta_4_1:
    sql(mi_consulta_4_1)
else:
    print("✍️ Escribe tu consulta")
    print("💡 PISTA: INNER JOIN productos p con categorias c ON p.categoria_id = c.id")
    print("💡 Usa alias: p.nombre AS nombre_producto, c.nombre AS nombre_categoria")

In [ ]:
# Solución 4.1
solucion_4_1 = """
SELECT
    p.nombre AS nombre_producto,
    p.precio,
    c.nombre AS nombre_categoria
FROM productos p
INNER JOIN categorias c ON p.categoria_id = c.id
"""
sql(solucion_4_1, titulo="✅ SOLUCIÓN EJERCICIO 4.1")

### Ejercicio 4.2: Ventas con nombre del cliente

In [ ]:
# EJERCICIO 4.2: Muestra ventas con el nombre del cliente
# Columnas: nombre_cliente, apellido_cliente, fecha_venta, cantidad

print("📝 EJERCICIO 4.2: Ventas con nombre del cliente")

mi_consulta_4_2 = ""

if mi_consulta_4_2:
    sql(mi_consulta_4_2)
else:
    print("✍️ Escribe tu consulta")
    print("💡 PISTA: JOIN ventas v con clientes cl ON v.cliente_id = cl.id")

In [ ]:
# Solución 4.2
solucion_4_2 = """
SELECT
    cl.nombre AS nombre_cliente,
    cl.apellido AS apellido_cliente,
    v.fecha_venta,
    v.cantidad
FROM ventas v
INNER JOIN clientes cl ON v.cliente_id = cl.id
"""
sql(solucion_4_2, titulo="✅ SOLUCIÓN EJERCICIO 4.2")

### Ejercicio 4.3: JOIN de 3 tablas - Ventas completas

In [ ]:
# EJERCICIO 4.3: Muestra ventas con cliente y producto (3 tablas)
# Columnas: nombre_cliente, nombre_producto, fecha_venta, cantidad, precio_unitario

print("📝 EJERCICIO 4.3: Ventas completas (cliente + producto)")

mi_consulta_4_3 = ""

if mi_consulta_4_3:
    sql(mi_consulta_4_3)
else:
    print("✍️ Escribe tu consulta")
    print("💡 PISTA: Necesitas 2 JOINs")
    print("   FROM ventas v")
    print("   JOIN clientes cl ON v.cliente_id = cl.id")
    print("   JOIN productos p ON v.producto_id = p.id")

In [ ]:
# Solución 4.3
solucion_4_3 = """
SELECT
    cl.nombre AS nombre_cliente,
    p.nombre AS nombre_producto,
    v.fecha_venta,
    v.cantidad,
    v.precio_unitario
FROM ventas v
INNER JOIN clientes cl ON v.cliente_id = cl.id
INNER JOIN productos p ON v.producto_id = p.id
"""
sql(solucion_4_3, titulo="✅ SOLUCIÓN EJERCICIO 4.3")

## 🎯 PARTE B: JOINs CON CÁLCULOS

### Ejercicio 4.4: Ventas con total calculado

In [ ]:
# EJERCICIO 4.4: Ventas con total (cantidad × precio_unitario)
# Columnas: nombre_cliente, nombre_producto, cantidad, precio_unitario, total

print("📝 EJERCICIO 4.4: Ventas con total calculado")

mi_consulta_4_4 = ""

if mi_consulta_4_4:
    sql(mi_consulta_4_4)
else:
    print("✍️ Escribe tu consulta")
    print("💡 PISTA: (v.cantidad * v.precio_unitario) AS total")

In [ ]:
# Solución 4.4
solucion_4_4 = """
SELECT
    cl.nombre AS nombre_cliente,
    p.nombre AS nombre_producto,
    v.cantidad,
    v.precio_unitario,
    (v.cantidad * v.precio_unitario) AS total
FROM ventas v
INNER JOIN clientes cl ON v.cliente_id = cl.id
INNER JOIN productos p ON v.producto_id = p.id
"""
sql(solucion_4_4, titulo="✅ SOLUCIÓN EJERCICIO 4.4")

### Ejercicio 4.5: Clientes con total gastado (GROUP BY)

In [ ]:
# EJERCICIO 4.5: Total gastado por cada cliente
# Columnas: nombre_cliente, email, total_gastado

print("📝 EJERCICIO 4.5: Total gastado por cliente")

mi_consulta_4_5 = ""

if mi_consulta_4_5:
    sql(mi_consulta_4_5)
else:
    print("✍️ Escribe tu consulta")
    print("💡 PISTA: SUM(v.cantidad * v.precio_unitario) + GROUP BY cliente")

In [ ]:
# Solución 4.5
solucion_4_5 = """
SELECT
    cl.nombre AS nombre_cliente,
    cl.email,
    SUM(v.cantidad * v.precio_unitario) AS total_gastado
FROM clientes cl
INNER JOIN ventas v ON cl.id = v.cliente_id
GROUP BY cl.id, cl.nombre, cl.email
ORDER BY total_gastado DESC
"""
sql(solucion_4_5, titulo="✅ SOLUCIÓN EJERCICIO 4.5")

## 🎯 PARTE C: LEFT JOIN BÁSICO

### Ejercicio 4.6: Todos los clientes (con y sin compras)

In [ ]:
# EJERCICIO 4.6: Todos los clientes con sus compras (incluir los que no han comprado)
# Columnas: nombre_cliente, apellido, fecha_venta, cantidad

print("📝 EJERCICIO 4.6: Todos los clientes (LEFT JOIN)")

mi_consulta_4_6 = ""

if mi_consulta_4_6:
    sql(mi_consulta_4_6)
else:
    print("✍️ Escribe tu consulta")
    print("💡 PISTA: LEFT JOIN para incluir clientes sin ventas")
    print("💡 Algunos clientes aparecerán con NULL en fecha_venta")

In [ ]:
# Solución 4.6
solucion_4_6 = """
SELECT
    cl.nombre AS nombre_cliente,
    cl.apellido,
    v.fecha_venta,
    v.cantidad
FROM clientes cl
LEFT JOIN ventas v ON cl.id = v.cliente_id
ORDER BY cl.nombre
"""
sql(solucion_4_6, titulo="✅ SOLUCIÓN EJERCICIO 4.6")

### Ejercicio 4.7: Clientes que NO han comprado

In [ ]:
# EJERCICIO 4.7: Encontrar clientes que NO han realizado compras
# Columnas: nombre_cliente, apellido, email, ciudad

print("📝 EJERCICIO 4.7: Clientes sin compras")

mi_consulta_4_7 = ""

if mi_consulta_4_7:
    sql(mi_consulta_4_7)
else:
    print("✍️ Escribe tu consulta")
    print("💡 PISTA: LEFT JOIN + WHERE v.cliente_id IS NULL")

In [ ]:
# Solución 4.7
solucion_4_7 = """
SELECT
    cl.nombre AS nombre_cliente,
    cl.apellido,
    cl.email,
    cl.ciudad
FROM clientes cl
LEFT JOIN ventas v ON cl.id = v.cliente_id
WHERE v.cliente_id IS NULL
"""
sql(solucion_4_7, titulo="✅ SOLUCIÓN EJERCICIO 4.7")

---

# 🏆 EJERCICIO FINAL INTEGRADOR
**Combina todo: SELECT + WHERE + ORDER BY + JOINs + GROUP BY**

## 🎯 Dashboard de Análisis Completo

In [ ]:
# EJERCICIO FINAL: Reporte de ventas por categoría
# Crear un reporte que muestre:
# - Nombre de la categoría
# - Total de productos en esa categoría
# - Total de ingresos generados
# - Número de ventas realizadas
# - Producto más caro de la categoría
# - Cliente que más compró de esa categoría
# Ordenado por total de ingresos (mayor a menor)

print("🏆 EJERCICIO FINAL: Dashboard por categorías")
print("Este es un desafío que combina TODO lo aprendido")

mi_consulta_final = ""

if mi_consulta_final:
    sql(mi_consulta_final, titulo="Mi dashboard")
else:
    print("\n📝 INSTRUCCIONES:")
    print("1. Une las 4 tablas (categorias, productos, ventas, clientes)")
    print("2. Agrupa por categoría")
    print("3. Calcula métricas con funciones de agregación")
    print("4. Ordena por ingresos totales")
    print("\n💡 ESTRUCTURA SUGERIDA:")
    print("   SELECT categoria, COUNT(productos), SUM(ingresos), etc.")
    print("   FROM categorias c")
    print("   JOIN productos p ON ...")
    print("   JOIN ventas v ON ...")
    print("   JOIN clientes cl ON ...")
    print("   GROUP BY categoria")
    print("   ORDER BY ingresos DESC")

In [ ]:
# Solución simplificada del ejercicio final
print("✅ SOLUCIÓN FINAL (Versión Simplificada):")

solucion_final = """
SELECT
    c.nombre AS categoria,
    COUNT(DISTINCT p.id) AS total_productos,
    COUNT(v.id) AS numero_ventas,
    SUM(v.cantidad * v.precio_unitario) AS ingresos_totales,
    ROUND(AVG(v.cantidad * v.precio_unitario), 2) AS venta_promedio,
    MAX(p.precio) AS producto_mas_caro
FROM categorias c
LEFT JOIN productos p ON c.id = p.categoria_id
LEFT JOIN ventas v ON p.id = v.producto_id
LEFT JOIN clientes cl ON v.cliente_id = cl.id
GROUP BY c.id, c.nombre
ORDER BY ingresos_totales DESC
"""

sql(solucion_final, titulo="Dashboard por categorías")

---

# 📊 VISUALIZACIÓN DE RESULTADOS

¡Vamos a crear gráficos con nuestros JOINs!

In [ ]:
# Crear visualizaciones de los resultados de JOINs

# Gráfico 1: Ventas por categoría
df_categorias = sql("""
SELECT
    c.nombre AS categoria,
    SUM(v.cantidad * v.precio_unitario) AS ingresos
FROM categorias c
LEFT JOIN productos p ON c.id = p.categoria_id
LEFT JOIN ventas v ON p.id = v.producto_id
GROUP BY c.nombre
ORDER BY ingresos DESC
""", False)

# Gráfico 2: Top clientes
df_clientes = sql("""
SELECT
    cl.nombre AS cliente,
    SUM(v.cantidad * v.precio_unitario) AS total_gastado
FROM clientes cl
INNER JOIN ventas v ON cl.id = v.cliente_id
GROUP BY cl.nombre
ORDER BY total_gastado DESC
LIMIT 8
""", False)

if df_categorias is not None and df_clientes is not None:
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

    # Gráfico 1: Ingresos por categoría
    df_categorias_filtrado = df_categorias.dropna()
    ax1.bar(df_categorias_filtrado['categoria'], df_categorias_filtrado['ingresos'],
           color='skyblue', alpha=0.8)
    ax1.set_title('💰 Ingresos por Categoría', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Categoría')
    ax1.set_ylabel('Ingresos (€)')
    ax1.tick_params(axis='x', rotation=45)

    # Gráfico 2: Top clientes
    ax2.barh(df_clientes['cliente'], df_clientes['total_gastado'], color='lightcoral')
    ax2.set_title('🏆 Top Clientes por Gasto', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Total Gastado (€)')

    # Gráfico 3: Distribución de ventas por vendedor
    df_vendedores = sql("""
    SELECT vendedor, COUNT(*) AS num_ventas
    FROM ventas
    GROUP BY vendedor
    """, False)

    ax3.pie(df_vendedores['num_ventas'], labels=df_vendedores['vendedor'],
           autopct='%1.1f%%', startangle=90)
    ax3.set_title('👥 Ventas por Vendedor', fontsize=14, fontweight='bold')

    # Gráfico 4: Evolución de ventas por mes
    df_mensual = sql("""
    SELECT
        strftime('%Y-%m', fecha_venta) AS mes,
        SUM(cantidad * precio_unitario) AS ingresos_mes
    FROM ventas
    GROUP BY strftime('%Y-%m', fecha_venta)
    ORDER BY mes
    """, False)

    ax4.plot(df_mensual['mes'], df_mensual['ingresos_mes'],
            marker='o', linewidth=2, markersize=8, color='green')
    ax4.set_title('📈 Evolución de Ventas Mensual', fontsize=14, fontweight='bold')
    ax4.set_xlabel('Mes')
    ax4.set_ylabel('Ingresos (€)')
    ax4.tick_params(axis='x', rotation=45)
    ax4.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

print("📊 ¡Visualizaciones creadas usando JOINs complejos!")
print("💡 Estos gráficos muestran el poder de combinar SQL con Python para análisis")

---

# 🎯 EVALUACIÓN Y RESUMEN

## 📊 Test Rápido: ¿Dominas ORDER BY y JOINs?

In [ ]:
print("🎯 TEST RÁPIDO ORDER BY Y JOINS")
print("="*50)
print("\n📝 Intenta resolver estas preguntas sin mirar atrás:")
print("\n1. Muestra los 3 clientes más jóvenes (nombre, edad)")
print("2. ¿Cuál es el producto más vendido? (nombre, total_unidades)")
print("3. Clientes de Madrid con sus compras totales")
print("4. Productos de 'Electrónicos' ordenados por precio")
print("5. ¿Qué cliente ha gastado más dinero? (nombre, total)")
print("\n💡 Cada pregunta combina ORDER BY con JOINs")
print("\n🎯 Criterio de éxito:")
print("   ✅ 5/5: ¡Master de SQL!")
print("   ✅ 3-4: Muy bueno, sigue practicando")
print("   ✅ 1-2: Repasa los conceptos básicos")
print("   ✅ 0: Necesitas más práctica")

In [ ]:
# Respuestas del test rápido
print("✅ RESPUESTAS DEL TEST RÁPIDO:")
print("="*50)

print("\n1. Los 3 clientes más jóvenes:")
sql("SELECT nombre, edad FROM clientes ORDER BY edad LIMIT 3")

print("\n2. Producto más vendido:")
sql("""
SELECT p.nombre, SUM(v.cantidad) AS total_unidades
FROM productos p
JOIN ventas v ON p.id = v.producto_id
GROUP BY p.nombre
ORDER BY total_unidades DESC
LIMIT 1
""")

print("\n3. Clientes de Madrid con compras:")
sql("""
SELECT
    cl.nombre,
    SUM(v.cantidad * v.precio_unitario) AS total_gastado
FROM clientes cl
JOIN ventas v ON cl.id = v.cliente_id
WHERE cl.ciudad = 'Madrid'
GROUP BY cl.nombre
ORDER BY total_gastado DESC
""")

In [ ]:
print("\n4. Electrónicos por precio:")
sql("""
SELECT p.nombre, p.precio
FROM productos p
JOIN categorias c ON p.categoria_id = c.id
WHERE c.nombre = 'Electrónicos'
ORDER BY p.precio DESC
""")

print("\n5. Cliente que más ha gastado:")
sql("""
SELECT
    cl.nombre,
    SUM(v.cantidad * v.precio_unitario) AS total_gastado
FROM clientes cl
JOIN ventas v ON cl.id = v.cliente_id
GROUP BY cl.nombre
ORDER BY total_gastado DESC
LIMIT 1
""")

---

# 🎉 ¡FELICITACIONES!

## 🏆 Has Dominado ORDER BY y JOINs Básicos

### ✅ Lo que ahora DOMINAS:

#### **📊 ORDER BY Experto:**
- ✅ Ordenar por una o múltiples columnas
- ✅ Usar ASC y DESC apropiadamente  
- ✅ Combinar ORDER BY con WHERE y LIMIT
- ✅ Obtener TOP N resultados
- ✅ Entender el orden de ejecución SQL

#### **🔗 JOINs Competente:**
- ✅ INNER JOIN entre 2 tablas
- ✅ INNER JOIN entre 3+ tablas
- ✅ LEFT JOIN para incluir registros sin coincidencias
- ✅ Encontrar registros que NO tienen relaciones
- ✅ Combinar JOINs con WHERE, ORDER BY, GROUP BY
- ✅ Calcular métricas agregadas con JOINs
- ✅ Usar alias de tabla correctamente (cl, p, v, c)

#### **🧩 Integración Avanzada:**
- ✅ Resolver problemas de negocio complejos
- ✅ Crear reportes ejecutivos con múltiples métricas
- ✅ Combinar SELECT + WHERE + JOIN + GROUP BY + ORDER BY
- ✅ Escribir consultas legibles y bien estructuradas
- ✅ Interpretar resultados para toma de decisiones

### 📊 **TUS LOGROS:**
- 🎯 **45+ ejercicios completados** (ORDER BY + JOINs)
- 📊 **4 visualizaciones** creadas con datos de JOINs
- 🏆 **1 ejercicio final** de nivel medio completado
- 🎓 **Test de evaluación** realizado

### 🚀 **Siguiente nivel - SQL Intermedio:**

Ya estás listo para conceptos más avanzados:

1. **Subconsultas complejas** - SELECT dentro de WHERE/FROM
2. **CTEs (Common Table Expressions)** - Consultas más legibles
3. **Window Functions** - RANK(), ROW_NUMBER(), LAG(), LEAD()
4. **CASE WHEN** - Lógica condicional avanzada
5. **Funciones de fecha y texto** - Manipulación avanzada de datos

### 💼 **Aplicaciones profesionales:**

Con lo que sabes ahora puedes:

- ✅ **Crear dashboards** básicos para empresas
- ✅ **Generar reportes** de ventas, clientes, inventario
- ✅ **Analizar tendencias** temporales y por categorías
- ✅ **Identificar insights** de negocio con datos
- ✅ **Colaborar** con equipos de análisis de datos

### 🎯 **Tu próxima misión:**

**Aplica estos conocimientos en un proyecto real:**
1. Encuentra un dataset público interesante
2. Importa los datos a una base de datos
3. Crea 5 preguntas de negocio
4. Respóndelas usando JOINs y ORDER BY
5. Visualiza los resultados

### 💪 **Mensaje final:**

**¡Has completado exitosamente el repaso de ORDER BY y JOINs! Ahora tienes las herramientas para conectar información de múltiples tablas y crear análisis significativos. Esto te convierte en un analista de datos capaz de resolver problemas reales de negocio.**

**¡Sigue practicando, sigue creciendo, y sigue descubriendo insights valiosos en los datos! 🚀📊✨**

---

### 📚 **Recursos para continuar:**
- **Mode Analytics SQL**: https://mode.com/sql-tutorial/ (JOINs avanzados)
- **W3Schools SQL**: https://www.w3schools.com/sql/ (referencia completa)
- **SQLBolt**: https://sqlbolt.com/ (práctica interactiva)
- **Kaggle Learn**: https://www.kaggle.com/learn/intro-to-sql (datasets reales)

**¡Tu journey en SQL apenas comienza! 🌟**